## Ticket Classification Prediction
The solution handles the misclassification of tickets using multifactor AI/ML based classification model. The As-Is classification scheme leads to higher MTTR(Mean Time to Resolution) and low FCR (First Call Resolution) due to misclassification of tickets. The current scheme handles these problems by automating classification of tickets using classification models. The solution considers factors such as ticket impact, urgency, priority along with ticket description and ticket categories.

### Contents

1. [Set up the environment](#Set-up-the-environment)
1. [Usage Instructions](#Usage-Instructions)
1. [Upload the data for training](#Upload-the-data-for-training)
1. [Run Training Job](#Run-Training-Job)
1. [Live Inference Endpoint](#Live Inference)
1. [Batch Transform Job](#Batch-Transform-Job)
1. [Output Interpretation](#Output-Interpretation)



<img src="images/Flow_diagram.JPG">

### Prerequisite

To run this algorithm you need to have access to the following AWS Services:
- Access to AWS SageMaker and the model package.
- An S3 bucket to specify input/output.
- Role for AWS SageMaker to access input/output from S3.

### Input format
#### Input:
Name of the file: <b>train.csv</b><br>
This file contains historical incidents that have been resolved. The solution uses the following incident specific inputs to derive specific productivity measures such as efficiency, experience and workload management across incident types for incident managers to make the predictions.<br><br>

</ul>
<li>  ID: Unique identifier for the request- alphanumeric e.g. INC0001029696</li>
<li> Reported_Day: The day of the week in number (Preferred format: 1-7)</li>

<li> prod_cat: First level category for requests e.g. Miscellaneous_Instance_Database_SQL Server Database</li>
<li> Country: Country of origin of request, Preferred format: USA)</li>
<li> Detailed_Description: Free Text Describing the problem in users works</li>
<li> Priority: Status of the request e.g. Low/Medium/High</li>
<li> Impact: High/Medium/Low
</ul><br>
NOTE:
</ul>
<li>Not all requests are mandatory. Optional Fields :Prod_Cat, Detailed_Description,prod_cat</li>

</ul>




## Set up the environment
Here we specify a bucket to use and the role that will be used for working with SageMaker.

In [29]:
# S3 prefix
prefix = 'churn-telecom'

# Define IAM role
import boto3
import re

import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()

## Create the session
The session remembers our connection parameters to SageMaker. We'll use it to perform all of our SageMaker operations.

In [30]:
import sagemaker as sage
from time import gmtime, strftime

sess = sage.Session()

## Upload the data for training
When training large models with huge amounts of data, you'll typically use big data tools, like Amazon Athena, AWS Glue, or Amazon EMR, to create your data in S3. For the purposes of this example, we're using classification dataset, which we have included.

We can use use the tools provided by the SageMaker Python SDK to upload the data to a default bucket.

In [31]:
data_location= 's3://mphasis-marketplace/churn-prediction-telecom/input/train.csv'

## Create an estimator and fit the model
In order to use SageMaker to fit our algorithm, we'll create an Estimator that defines how to use the container to train. This includes the configuration we need to invoke SageMaker training:
- The container name. This is constructed as in the shell commands above.
- The role. As defined above.
- The instance count which is the number of machines to use for training.
- The instance type which is the type of machine to use for training.
- The output path determines where the model artifact will be written.
- The session is the SageMaker session object that we defined above

Then we use fit() on the estimator to train against the data that we uploaded above.

In [32]:
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
image = '{}.dkr.ecr.{}.amazonaws.com/telecom-pycaret-churn'.format(account, region)

tree = sage.estimator.Estimator(image,
                       role, 3, 'ml.c4.2xlarge',
                      output_path="s3://{}/output".format(sess.default_bucket()),
                       sagemaker_session=sess)

tree.fit(data_location)

2021-04-20 15:21:42 Starting - Starting the training job...
2021-04-20 15:21:44 Starting - Launching requested ML instancesProfilerReport-1618932102: InProgress
......
2021-04-20 15:23:07 Starting - Preparing the instances for training......
2021-04-20 15:24:12 Downloading - Downloading input data
2021-04-20 15:24:12 Training - Downloading the training image.......Starting the training.
(5986, 19)
IntProgress(value=0, description='Processing: ', max=3)
                                                                    
                                                                    
Initiated  . . . . . . . . . . . . . . . . . .              15:25:12
Status     . . . . . . . . . . . . . . . . . .  Loading Dependencies
                                                                           
                                                                           
Initiated  . . . . . . . . . . . . . . . . . .                     15:25:12
Status     . . . . . . . . . . . . . . 

## Hosting your model
You can use a trained model to get real time predictions using HTTP endpoint. Follow these steps to walk you through the process.


In [33]:
training_job_name = tree.latest_training_job.name
attached_tree = sage.estimator.Estimator.attach(training_job_name)



2021-04-20 15:26:28 Starting - Preparing the instances for training
2021-04-20 15:26:28 Downloading - Downloading input data
2021-04-20 15:26:28 Training - Training image download completed. Training in progress.
2021-04-20 15:26:28 Uploading - Uploading generated training model
2021-04-20 15:26:28 Completed - Training job completed



### Deploy the model
Deploying the model to SageMaker hosting just requires a deploy call on the fitted model. This call takes an instance count, instance type, and optionally serializer and deserializer functions. These are used when the resulting predictor is created on the endpoint.

In [36]:

from sagemaker.predictor import csv_serializer
predictor = attached_tree.deploy(4, 'ml.m4.xlarge', serializer=csv_serializer,endpoint_name='churn-telecom-pycaret')

-------------!

## Choose some data and use it for a prediction


In [37]:
test_data  = 's3://mphasis-marketplace/churn-prediction-telecom/input/test.csv'

data = pd.read_csv(test_data,encoding='ISO-8859–1',header=None)
data

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,NaN,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
1,1869.0,7010-BRBUU,Male,0,Yes,Yes,72,Yes,Yes,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Credit card (automatic),24.1,1734.65
2,4528.0,9688-YGXVR,Female,0,No,No,44,Yes,No,Fiber optic,...,Yes,Yes,No,Yes,No,Month-to-month,Yes,Credit card (automatic),88.15,3973.2
3,6344.0,9286-DOJGF,Female,1,Yes,No,38,Yes,Yes,Fiber optic,...,No,No,No,No,No,Month-to-month,Yes,Bank transfer (automatic),74.95,2869.85
4,6739.0,6994-KERXL,Male,0,No,No,4,Yes,No,DSL,...,No,No,No,No,Yes,Month-to-month,Yes,Electronic check,55.9,238.5
5,432.0,2181-UAESM,Male,0,No,No,2,Yes,No,DSL,...,No,Yes,No,No,No,Month-to-month,No,Electronic check,53.45,119.5
6,2215.0,4312-GVYNH,Female,0,Yes,No,70,No,No phone service,DSL,...,No,Yes,Yes,No,Yes,Two year,Yes,Bank transfer (automatic),49.85,3370.2
7,5260.0,2495-KZNFB,Female,0,No,No,33,Yes,Yes,Fiber optic,...,No,No,No,No,Yes,Month-to-month,Yes,Electronic check,90.65,2989.6
8,6001.0,4367-NHWMM,Female,0,No,No,1,No,No phone service,DSL,...,No,No,No,No,No,Month-to-month,Yes,Mailed check,24.9,24.9
9,1480.0,8898-KASCD,Male,0,No,No,39,No,No phone service,DSL,...,No,Yes,Yes,No,No,One year,No,Mailed check,35.55,1309.15


In [38]:
predictions = predictor.predict(data.values).decode('utf-8')



The csv_serializer has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [39]:
print(predictions)

nan,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Label,Score
1869.0,7010-BRBUU,Male,0,Yes,Yes,72,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Credit card (automatic),24.1,1734.65,No,0.9858
4528.0,9688-YGXVR,Female,0,No,No,44,Yes,No,Fiber optic,No,Yes,Yes,No,Yes,No,Month-to-month,Yes,Credit card (automatic),88.15,3973.2,No,0.7459
6344.0,9286-DOJGF,Female,1,Yes,No,38,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Bank transfer (automatic),74.95,2869.85,No,0.5653
6739.0,6994-KERXL,Male,0,No,No,4,Yes,No,DSL,No,No,No,No,No,Yes,Month-to-month,Yes,Electronic check,55.9,238.5,Yes,0.6318
432.0,2181-UAESM,Male,0,No,No,2,Yes,No,DSL,Yes,No,Yes,No,No,No,Month-to-month,No,Electronic check

### Output

Output files contains column predicted Group, which has the predicted class

In [40]:
transform_output_folder = "batch-transform-output"
output_path="s3://{}/{}".format(sess.default_bucket(), transform_output_folder)

transformer = tree.transformer(instance_count=1,
                               instance_type='ml.m4.xlarge',
                               output_path=output_path)

In [41]:
transformer.transform(test_data, content_type='text/csv')
transformer.wait()
print("Batch Transform output saved to " + transformer.output_path)

.............................Starting the inference server with 4 workers.
[2021-04-20 15:39:52 +0000] [12] [INFO] Starting gunicorn 20.1.0
[2021-04-20 15:39:52 +0000] [12] [INFO] Listening at: unix:/tmp/gunicorn.sock (12)
[2021-04-20 15:39:52 +0000] [12] [INFO] Using worker: gevent
[2021-04-20 15:39:52 +0000] [16] [INFO] Booting worker with pid: 16
[2021-04-20 15:39:52 +0000] [17] [INFO] Booting worker with pid: 17
[2021-04-20 15:39:52 +0000] [18] [INFO] Booting worker with pid: 18
[2021-04-20 15:39:52 +0000] [19] [INFO] Booting worker with pid: 19
Transformation Pipeline and Model Successfully Loaded
169.254.255.130 - - [20/Apr/2021:15:40:00 +0000] "GET /ping HTTP/1.1" 200 1 "-" "Go-http-client/1.1"
169.254.255.130 - - [20/Apr/2021:15:40:00 +0000] "GET /execution-parameters HTTP/1.1" 404 2 "-" "Go-http-client/1.1"
Transformation Pipeline and Model Successfully Loaded
169.254.255.130 - - [20/Apr/2021:15:40:01 +0000] "POST /invocations HTTP/1.1" 200 1684 "-" "Go-http-client/1.1"

2021-

#### Inspect the Batch Transform Output in S3

In [42]:
from urllib.parse import urlparse

parsed_url = urlparse(transformer.output_path)
bucket_name = parsed_url.netloc
file_key = '{}/{}.out'.format(parsed_url.path[1:], "test.csv")



s3_client = sess.boto_session.client('s3')

response = s3_client.get_object(Bucket = sess.default_bucket(), Key = file_key)
response_bytes = response['Body'].read().decode('utf-8')
print(response_bytes)

Unnamed: 0,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Label,Score
1869,7010-BRBUU,Male,0,Yes,Yes,72,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Credit card (automatic),24.1,1734.65,No,0.9858
4528,9688-YGXVR,Female,0,No,No,44,Yes,No,Fiber optic,No,Yes,Yes,No,Yes,No,Month-to-month,Yes,Credit card (automatic),88.15,3973.2,No,0.7459
6344,9286-DOJGF,Female,1,Yes,No,38,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Bank transfer (automatic),74.95,2869.85,No,0.5653
6739,6994-KERXL,Male,0,No,No,4,Yes,No,DSL,No,No,No,No,No,Yes,Month-to-month,Yes,Electronic check,55.9,238.5,Yes,0.6318
432,2181-UAESM,Male,0,No,No,2,Yes,No,DSL,Yes,No,Yes,No,No,No,Month-to-month,No,Electronic check,53

### View Output
Lets read results of above transform job from s3 files and print output

In [43]:
s3_client = sess.boto_session.client('s3')
s3_client.download_file(sess.default_bucket(), "{}/test.csv.out".format(transform_output_folder), '/tmp/test.csv.out')
with open('/tmp/test.csv.out') as f:
    results = f.readlines() 
##print("Transform results: \n{}".format(''.join(results)))
string_final = ''.join(results)

print(string_final)

with open("Output.txt", "w") as text_file:
    text_file.write(string_final)

Unnamed: 0,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Label,Score
1869,7010-BRBUU,Male,0,Yes,Yes,72,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Credit card (automatic),24.1,1734.65,No,0.9858
4528,9688-YGXVR,Female,0,No,No,44,Yes,No,Fiber optic,No,Yes,Yes,No,Yes,No,Month-to-month,Yes,Credit card (automatic),88.15,3973.2,No,0.7459
6344,9286-DOJGF,Female,1,Yes,No,38,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Bank transfer (automatic),74.95,2869.85,No,0.5653
6739,6994-KERXL,Male,0,No,No,4,Yes,No,DSL,No,No,No,No,No,Yes,Month-to-month,Yes,Electronic check,55.9,238.5,Yes,0.6318
432,2181-UAESM,Male,0,No,No,2,Yes,No,DSL,Yes,No,Yes,No,No,No,Month-to-month,No,Electronic check,53